<a href="https://colab.research.google.com/github/Bigote93/AnalisisTwitters/blob/main/AnalisisTwitters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Aplicar el llamado de almacenamiento

from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
pip install scikit-learn

## Carga de recursos de datos

In [9]:
twitters = pd.read_csv('/content/drive/MyDrive/DocumentosModuloIV/AnalisisTwitter/colombian_elections.csv')
twitters.head(5)

,Unnamed: 0,tweet_id,tweetText,polarity_value,polarity_type,topic,set
0,2,ID:142391947707940864,@marodriguezb Gracias MAR,P,AGREEMENT,otros,train
1,3,ID:142416095012339712,"Off pensando en el regalito Sinde, la que se v...",N,AGREEMENT,política,train
2,4,ID:142422495721562112,Conozco a alguien q es adicto al drama! Ja ja ...,P,AGREEMENT,otros,train
3,6,ID:142483342040907776,Toca @crackoviadeTV3 . Grabación dl especial N...,P,AGREEMENT,entretenimiento,train
4,8,ID:142494476051562496,Buen día todos! Lo primero mandar un abrazo gr...,P,AGREEMENT,otros,train


## Modificacion de tipo de dato y columnas de analisis

Seleccionaremos las columnas de sentimientos que analizaremos en este cuaderno.
> Por lo tanto debemos transformar las columnas a nombres en español mas facilmente descriptivos
> Cambiemos los valores de la columna sentimientos a binarios

* tweet_id : IDTwitter
* tweetText: tweetText
* polarity_value: sentimiento
* polarity_type: sentimientoTipo
* topic: asunto
* set: conjunto

In [10]:
#Transformar el nombre de columnas
mapa_columnas = {
    'polarity_value': 'sentimiento',
    'polarity_type': 'sentimientoTipo',
    'topic': 'asunto',
    'set': 'conjunto'
}

#Cambiar nombre de columnas
twitters.rename(columns = mapa_columnas, inplace = True)
twitters.head(5)

,Unnamed: 0,tweet_id,tweetText,sentimiento,sentimientoTipo,asunto,conjunto
0,2,ID:142391947707940864,@marodriguezb Gracias MAR,P,AGREEMENT,otros,train
1,3,ID:142416095012339712,"Off pensando en el regalito Sinde, la que se v...",N,AGREEMENT,política,train
2,4,ID:142422495721562112,Conozco a alguien q es adicto al drama! Ja ja ...,P,AGREEMENT,otros,train
3,6,ID:142483342040907776,Toca @crackoviadeTV3 . Grabación dl especial N...,P,AGREEMENT,entretenimiento,train
4,8,ID:142494476051562496,Buen día todos! Lo primero mandar un abrazo gr...,P,AGREEMENT,otros,train


## Matriz de tokens

Creamos la matriz de ejemplo de tokens por columna

* Creamos un conjunto de datos de ejemplo
* Creamos una instancia de vectorizador
* Cargamos la instancia con el texto de ejemplo
* Creamos la matriz parseada, es decir una matriz que identifique 0 y 1 para verificar si la palabra se encuentra o no en la oracion del dataset "texto"

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

#Crear conjunto de datos

texto = [
    "Las comidas de la semana son bastante buenas",
    "La comida que prepara mi novia es muy buena",
    "El fin de semana nunca comemos comida buena"
]

#Crear vectorizador
vectorizador = CountVectorizer()

#Entrenar vectorizador
bag_of_words = vectorizador.fit_transform(texto)

#Matriz parceada
matriz = pd.DataFrame.sparse.from_spmatrix(
    bag_of_words,
    columns = vectorizador.get_feature_names_out()
)

matriz

,bastante,buena,buenas,comemos,comida,comidas,de,el,es,fin,la,las,mi,muy,novia,nunca,prepara,que,semana,son
0,1,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,1,1
1,0,1,0,0,1,0,0,0,1,0,1,0,1,1,1,0,1,1,0,0
2,0,1,0,1,1,0,1,1,0,1,0,0,0,0,0,1,0,0,1,0


## Trabajar los datos del dataset original

* Cargamos el dataset twitters y hacemos el cambio del nombre de las variables de la forma:
> - "N" : 0
> - "P" : 1

* Luego creamos los datos de entrenamiento con el dataset considerando las columnas de texto y sentimiento

In [14]:
from sklearn.model_selection import train_test_split

#Creamos una columna nueva con la asignacion de valores
twitters['Sentimiento_c'] = twitters['sentimiento'].replace(
    ['N','P'],[0,1]
)

#Instanciamos los conjuntos de datos de entrenamiento y test
datos_analisis = train_test_split(
    twitters.tweetText,
    twitters.Sentimiento_c,
    random_state= 20
)

#Asignamos valores de entrenamiento y test
X_train, X_test, y_train, y_test = datos_analisis


## Instanciamos el modelo regreesion

In [15]:
from sklearn.linear_model import LogisticRegression

#Instanciamos el modelo regression
regression_instance = LogisticRegression()

#Cargamos los datos al modelo de regression
regression_instance.fit(X_train, y_train)

#Creamos el score del modelo realizado utilizando los datos test
accuracy = regression_instance.score(X_test, y_test)
accuracy

ValueError: ignored